In [386]:
import json
from graphviz import Digraph
import re
from collections import namedtuple, defaultdict

# Shunting Yard Algo

In [387]:
def insert_caret_before_alpha(string):
    result = ""
    for i in range(len(string)):
        char = string[i]
        if char == '-' or char == '|':
            if i == len(string) - 1 or (i != len(string) - 1 and not (string[i + 1].isalpha() or string[i + 1].isdigit() or string[i + 1]=='.')):
                return False
            if i == 0 or (i != 0 and not (string[i - 1].isalpha() or string[i - 1].isdigit() or string[i - 1]=='.')):
                return False
        if (char.isalpha() or char.isdigit() or char=='.' or char=='[' or char=='('): #it's a charachter/dig/.
            if(i!=0):
               
                if(i>1):
                    if((string[i-1]=='+' or string[i-1]=='*' or string[i-1]=='?' ) and (string[i-2]==']' or string[i-2]==')' or  string[i-2].isalpha() or  string[i-2].isdigit() or  string[i-2]=='.')):
                        result += '^'
                        result += char
                    
                    elif(string[i-1].isalpha() or string[i-1].isdigit() or string[i-1]=='.' or string[i-1]=='*' or string[i-1]==']' or string[i-1]==')'): #it's not the first charachter
                        result += '^'
                        result += char
                    else:
                        result += char
                else:
                    if(string[i-1].isalpha() or string[i-1].isdigit() or string[i-1]=='.'): #it's not the first charachter
                        result += '^'
                        result += char
                    else:
                        result += char
           
            else:
                result += char
        
        else:
                result += char
    return result


def shuntingYardAlgo(infix):
    postfix = ""
    stack = []
    precedence = {'*': 5, '+': 4, '?': 3, '^': 2,'|': 1}
    opened=0
    infix=insert_caret_before_alpha(infix)
    if(infix == False):
        return False
    index = 0
    print("after adding ^ : " +infix)
    while index < len(infix):
        c = infix[index]
        if c == '(':
            stack.append(c)
            opened += 1
        elif c == ')':
            while stack[-1] != '(':
                postfix += stack.pop()
            if stack == []:
                return False
            stack.pop()
            opened -= 1
        elif c == '-':
                if(index == len(infix) - 1):
                    return False
                if not (infix[index + 1].isalpha() or infix[index + 1].isdigit()):
                    return False
        elif c in precedence:
            while stack and precedence.get(stack[-1], 0) >= precedence[c]:
                postfix += stack.pop()
            stack.append(c)
        elif c == '[':
            transition = "[" 
            index += 1  
            while index < len(infix) and infix[index] != ']':
                if infix[index] != '^':
                    transition += infix[index]
                index += 1  
            if index >= len(infix):
                return False 
            transition += ']' 
            postfix += transition  
        else:
            postfix += c

        index += 1 
    if opened != 0:
        return False
    while stack:
        postfix += stack.pop()
    return postfix

    



# Postfix to NFA

In [388]:

class State:
    def __init__(self, label):
        self.label = label
        self.edges = []

class Edge:
    def __init__(self, value, destination):
        self.value = value
        self.destination = destination

class NFA:
    def __init__(self, initial, accept, states):
        self.initial = initial
        self.accept = accept
        self.states = states
    
    def to_json(self):
        fsm = {}
        fsm["startingState"] = self.initial.label
        
        for state in self.states:
            fsm[state.label] = {"isTerminatingState": state == self.accept}
            edges_sorted = sorted(state.edges, key=lambda x: x.destination.label)
            for edge in edges_sorted:
                if len(edges_sorted) == 1:
                    fsm[state.label][edge.value] = edge.destination.label
                else:
                    destinations = [edge.destination.label for edge in edges_sorted if edge.value == edge.value]
                    fsm[state.label][edge.value] = destinations
                    
        return fsm


def MakeState(counter, val, stack):
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge(val, state2))
    nfa_made = NFA(state1, state2, [state1, state2])
    stack.append(nfa_made)

def zeroOrMore(counter, state, stack):
    nfa_made = stack.pop()
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge('ε', nfa_made.initial))
    state1.edges.append(Edge('ε', state2))
    nfa_made.accept.edges.append(Edge('ε', state2))
    nfa_made.accept.edges.append(Edge('ε', state1))
    nfa_final = NFA(state1, state2, [state1, state2] + nfa_made.states)
    stack.append(nfa_final)

def oneOrMore(counter, state, stack):
    nfa_made = stack.pop()
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge('ε', nfa_made.initial))
    nfa_made.accept.edges.append(Edge('ε', state2))
    nfa_made.accept.edges.append(Edge('ε', state1))
    nfa_final = NFA(state1, state2, [state1, state2] + nfa_made.states)
    stack.append(nfa_final)

def zeroOrOne(counter, state, stack):
    nfa_made = stack.pop()
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge('ε', nfa_made.initial))
    state1.edges.append(Edge('ε', state2))
    nfa_made.accept.edges.append(Edge('ε', state2))
    nfa_final = NFA(state1, state2, [state1, state2] + nfa_made.states)
    stack.append(nfa_final)

def concatenate(counter, state, stack):
    nfa_made2 = stack.pop()
    nfa_made1 = stack.pop()
    nfa_made1.accept.edges.append(Edge('ε', nfa_made2.initial))
    nfa_final = NFA(nfa_made1.initial, nfa_made2.accept, nfa_made1.states + nfa_made2.states)
    stack.append(nfa_final)

def oring(counter, state, stack):
    nfa_made2 = stack.pop()
    nfa_made1 = stack.pop()
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge('ε', nfa_made1.initial))
    state1.edges.append(Edge('ε', nfa_made2.initial))
    nfa_made1.accept.edges.append(Edge('ε', state2))
    nfa_made2.accept.edges.append(Edge('ε', state2))
    nfa_final = NFA(state1, state2, [state1, state2] + nfa_made1.states + nfa_made2.states)
    stack.append(nfa_final)


def thompsons(postfix):
    stack = []
    counter = 1
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.'
    digits = '0123456789'
    index = 0
    while index < len(postfix):
        c = postfix[index]
        if c in alphabet or c in digits:
            MakeState(counter, c, stack)
            counter += 2
        elif c == '*':
            zeroOrMore(counter, c, stack)
            counter += 2
        elif c == '+':
            oneOrMore(counter, c, stack)
            counter += 2
        elif c == '.':
            concatenate(counter, c, stack)
        elif c == '?':
            zeroOrOne(counter, c, stack)
            counter += 2
        elif c == '^':
            concatenate(counter, c, stack)
        elif c == '|':
            oring(counter, c, stack)
            counter += 2
        elif c == '[':
            transition = "[" + postfix[index + 1] 
            index += 2  
            while postfix[index] != ']':
                transition += postfix[index]
                index += 1  
            transition += postfix[index] 
            
            MakeState(counter, transition, stack)
            counter += 2
        
        index += 1  

    return stack.pop()




# DFA 


In [389]:
def add_epsilon_moves(queue: list,NFA) -> list:
    visited = []

    while queue:
        state = queue.pop(0) 
        visited.append(state)
        if 'ε' in NFA[state]:
            epsilon_value = NFA[state]['ε']
            if isinstance(epsilon_value, list):
                for item in epsilon_value:
                    if item not in visited and item not in queue:
                        queue.append(item)
            else:
                if epsilon_value not in visited and epsilon_value not in queue:
                    queue.append(epsilon_value)
    return visited

def get_possible_inputs(states: set, NFA,input_list=None) -> set:
    inputs = set()
    for state in states:
        keys = list(input_list[state].keys()) if input_list is not None else list(NFA[state].keys())
        for key in keys[1:]:
            if key != 'ε' and key not in inputs:
                inputs.add(key)
    return inputs

def add_inputs(states: set, inputs: set,NFA) -> dict :
    state_inputs = {}
    for input in inputs:
     
        ns = set()
        for state in states:
            if(input in NFA[state]):
                ns.add(NFA[state][input])
        ns.update(add_epsilon_moves(list(ns),NFA))
        state_inputs[input] = ns
    return state_inputs

def checkTerminating(states: set, NFA,input_list=None) -> bool:
    for state in states:
        state_dict = input_list[state] if input_list is not None else NFA[state]
        if state_dict['isTerminatingState']:
            return True
    return False

def sort_dict(d: dict):
    term_state = d.pop('isTerminatingState')
    sorted_dict = dict(sorted(d.items()))
    sorted_dict = {"isTerminatingState": term_state, **sorted_dict}
    return sorted_dict

# DFA minimization

# final

In [390]:

def TheBigFunction(infix):
    try:
     re.compile(infix)

    except re.error:
        print("Non valid regex pattern")
        return
        
    postfix = shuntingYardAlgo(infix)
    
    if(postfix==False):
        print("Invalid input")
    else:
        print("after shunting yard "+postfix)
        nfa = thompsons(postfix)
        json_data = nfa.to_json()
     
        with open('NFA.json', 'w') as _:
         json.dump(json_data, _, indent=4)


        graph = Digraph(graph_attr={'rankdir': 'LR'})
        graph.node('', shape='none')

        for key in json_data:
            if key != 'startingState':
                if json_data[key]["isTerminatingState"]:
                    graph.node(name=key, label=key, shape='doublecircle')
                else:
                    graph.node(name=key, label=key, shape='circle')

        for key in json_data:
            if key != 'startingState':
                for edge_value, destinations in json_data[key].items():
                    if edge_value != 'isTerminatingState':
                        if isinstance(destinations, list):
                            for destination in destinations:
                                if edge_value == 'ε':
                                    graph.edge(key, destination, label='ε')
                                else:
                                    graph.edge(key, destination, label=edge_value)
                        else:
                            destination = destinations
                            if edge_value == 'ε':
                                graph.edge(key, destination, label='ε')
                            else:
                                graph.edge(key, destination, label=edge_value)


        graph.edge('', json_data['startingState'], label='')

        graph.render('./NFA', view=True, format='png', cleanup=True)

        with open('NFA.json') as _:
            NFA = json.load(_)
        startingState = NFA['startingState']
        states = set(); states_dict = {}
        states.add(startingState)
        ct = 0; nname = f"NS{ct}"
        states.update(add_epsilon_moves([startingState],NFA))

        states_dict['startingState'] = nname
        states_dict[nname] = {'isTerminatingState': checkTerminating(states,NFA
        )}; 
        states_dict[nname]['states'] = states; ct+=1
        states_dict[nname]['inp'] = ', '.join(map(str, states))

        kv = list(states_dict.keys())
        i = 0
        while i < len(kv):
            ns = kv[i]
            ep_states = set().clear()
            if ns == "startingState":
                i += 1
                continue
            ep_states = add_epsilon_moves(list(states_dict[ns].get('states')),NFA)
            inputs = get_possible_inputs(ep_states,NFA)
            outStates = add_inputs(ep_states, inputs,NFA)
            for input in inputs:
                for key in list(states_dict.keys())[1:]:
                    if(states_dict[key].get('states') == outStates[input]):
                        nname = key
                        break
                else:   #Only enter if not breaks
                    nname = f"NS{ct}"
                    states_dict[nname] = {'isTerminatingState': checkTerminating(outStates[input],NFA)}
                    states_dict[nname]['states'] = outStates[input]
                    states_dict[nname]['inp'] = ', '.join(map(str, outStates[input]))
                    ct+=1
                states_dict[ns][input] = nname
            kv = list(states_dict.keys())
            i += 1
        cleaned_dict = {}
        cleaned_dict['startingState'] = states_dict['startingState']
        for key in states_dict:
            if key != "startingState":
                new_value = {k: v for k, v in states_dict[key].items() if (k != 'states' and k != 'inp')}
                cleaned_dict[key] = sort_dict(new_value)

        
        Min_list = [[key for key, value in cleaned_dict.items() if key != "startingState" and not value['isTerminatingState']],
                    [key for key, value in cleaned_dict.items() if key != "startingState" and value['isTerminatingState']]]

        list_rows = Min_list
        Row = namedtuple('Row', ['name','inputs', 'values'])

        while 1:
            state_to_index = {}
            for i, sublist in enumerate(list_rows):
                for state in sublist:
                    state_to_index[state] = i

            rows = []
            for lst in list_rows:
                for key in lst:
                    dict_state = cleaned_dict[key]
                    key_values = []; inputs = []; inputs_values = []

                    for k, v in dict_state.items():
                        if k != 'isTerminatingState':
                            inputs.append(k)
                            key_values.append(state_to_index.get(v))
                            # inputs_values.append(list(zip([k], [state_to_index.get(v)])))

                    row = Row(key, inputs, (state_to_index.get(key), key_values))
                    rows.append(row)

            dict_rows = defaultdict(list)
            for row in rows:
                dict_rows[(row.values[0], tuple(row.values[1]), tuple(row.inputs))].append(row.name)

            new_list_rows = list(map(list, dict_rows.values()))
            if new_list_rows == list_rows:
                break
            list_rows = new_list_rows

        state_to_index = {}
        for i, sublist in enumerate(list_rows):
            for state in sublist:
                state_to_index[state] = i

        fin_dict = {"startingState": f"S{state_to_index[cleaned_dict['startingState']]}"}
        for fin_list in list_rows:
            fin_terminating = checkTerminating(set(fin_list), NFA,cleaned_dict)
            for item in fin_list:
                k = f"S{state_to_index[item]}"
                ns = {'isTerminatingState': fin_terminating}
                ns.update({it: f"S{state_to_index[cleaned_dict[item][it]]}" for it in cleaned_dict[item] if it != "isTerminatingState"})
                fin_dict[k]=ns
                break

        with open('minimized_DFA.json', 'w') as _:
            json.dump(fin_dict, _, indent=4)

            
        graph = Digraph(graph_attr={'rankdir': 'LR'})
        graph.node('', shape='none')
        for key in fin_dict:
            if key != 'startingState':
                if fin_dict[key]["isTerminatingState"]:
                    graph.node(name= key, shape='doublecircle')
                else:
                    graph.node(name=key, shape='circle')

        for key in fin_dict:
            if key != 'startingState':
                for states in fin_dict[key]:
                    if states != 'isTerminatingState':
                        graph.edge(key,fin_dict[key][states],states)
        graph.edge('', fin_dict['startingState'])
        graph.unflatten().render('./minimized_dfa', view=True, format='png', cleanup=True)


        

        


    

In [393]:
infix = input()
TheBigFunction(infix)


after adding ^ : ((a|b|c)+^9|5^5?^(z^z^z)*)
after shunting yard ab|c|+9^55?^zz^z^*^|
